<a href="https://colab.research.google.com/github/BaekKyunShin/Kaggle-Competition-Including-Dacon/blob/master/bert_tokenizer_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [360]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [361]:
## 첫 실행시 주석 제거 후 transformers를 다운받으세요.
!pip install transformers

shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
The folder you are executing pip from can no longer be found.


In [362]:
import pandas as pd
import numpy as np

# 파이토치
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
from torch.nn import Conv2d, AdaptiveAvgPool2d, Linear
import torch.nn.functional as F

# 토크나이저
import transformers
from transformers import AutoTokenizer

# nltk
import nltk
from nltk.corpus import stopwords 

# 기타
import os
import re
import easydict

In [363]:
# 파라미터 설정
args = easydict.EasyDict({ 'chdir': '/content/gdrive/My Drive/colab/Dacon_Novel_Writer_Classification/',
                          'train_dir': 'open/train.csv', 
                          'test_dir': 'open/test_x.csv',
                          'submission_dir': 'open/sample_submission.csv',
                          'fianl_submission_dir': 'open/submission.csv',
                          'model_dir': 'ckpt/',
                          'max_len': 100,
                          'tokenizer': AutoTokenizer.from_pretrained('bert-base-uncased'),
                          'epochs': 10,
                          'learning_rate': 0.01,
                          'wd': 1e-5,
                          'batch_size': 64,
                          'train': True})

In [365]:
# device 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [366]:
#경로 설정
os.chdir(args.chdir)

In [367]:
# model_dir 경로 생성
if not os.path.isdir(args.model_dir) :
    os.makedirs(args.model_dir)

In [368]:
# 파일 불러오기
train = pd.read_csv(args.train_dir, encoding='utf-8')
test = pd.read_csv(args.test_dir, encoding='utf-8')
sample_submission = pd.read_csv(args.submission_dir, encoding='utf-8')

In [369]:
train.drop('index', axis=1, inplace=True)
train.head(2)

,text,author
0,"He was almost choking. There was so much, so m...",3
1,"“Your sister asked for it, I suppose?”",2


In [370]:
test.drop('index', axis=1, inplace=True)
test.head(2)

,text
0,“Not at all. I think she is one of the most ch...
1,"""No,"" replied he, with sudden consciousness, ""..."


In [371]:
sample_submission.head(2)

,index,0,1,2,3,4
0,0,0,0,0,0,0
1,1,0,0,0,0,0


# 전처리

In [372]:
# 불용어
basic_stopwords = { "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", 
             "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", 
             "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", 
             "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", 
             "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", 
             "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", 
             "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", 
             "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", 
             "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", 
             "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", 
             "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" }

nltk.download('stopwords')
nltk_stopwords = set(stopwords.words('english'))

final_stopwords = nltk_stopwords.union(basic_stopwords)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [373]:
# 부호를 제거해주는 함수
def alpha_num(text):
    return re.sub(r'[^A-Za-z0-9 ]', '', text)

# 불용어 제거해주는 함수
def remove_stopwords(text: str):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in final_stopwords:
            final_text.append(i.strip())
    return " ".join(final_text)
    
# 전처리 적용
train['text']=train['text'].apply(alpha_num)
test['text']=test['text'].apply(alpha_num)
train['text'] = train['text'].str.lower()
test['text'] = test['text'].str.lower()
train['text'] = train['text'].apply(alpha_num).apply(remove_stopwords)
test['text'] = test['text'].apply(alpha_num).apply(remove_stopwords)

In [374]:
def encode_text(text: str, tokenizer: AutoTokenizer, max_len: int):
    encoded_text = tokenizer(text, padding=True, truncation=True, max_length=max_len, return_tensors="pt")
    encoded_text_ids = encoded_text['input_ids'][0]
    padding_length = max_len - len(encoded_text_ids)
    if padding_length > 0:
        encoded_text_ids_pad = encoded_text_ids.tolist() + ([0] * padding_length )
        return encoded_text_ids_pad
    else:
        return encoded_text_ids

In [375]:
class TextDataset(Dataset):
    def __init__(self, df:pd.DataFrame, tokenizer: AutoTokenizer, max_len: int, mode: str='train'):
        self.df = df
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.mode = mode
        self.full_text = [x for x in self.df['text']]
        if self.mode == 'train':
            self.full_author = [x for x in self.df['author']]

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, idx: int):
        text = self.full_text[idx]
        encoded_text_ids = encode_text(text, self.tokenizer, self.max_len)

        if self.mode == 'train':    
            author = self.full_author[idx]
            return torch.tensor(encoded_text_ids), torch.tensor(author)
        else:
            return torch.tensor(encoded_text_ids)

In [376]:
# Dataset, Dataloader 정의
train_dataset = TextDataset(train, args.tokenizer, args.max_len, 'train')
test_dataset = TextDataset(test, args.tokenizer, args.max_len, 'test')

train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [377]:
num_texts = train.shape[0]
num_classses = len(train['author'].unique())

print('num of texts is ', num_texts, ', num of classes is ', num_classses)

num of texts is  54879 , num of classes is  5


In [378]:
input = (args.batch_size, 1, args.max_len)
conv1d = (1, 7, 3, 1)
n_output = np.trunc((input[2] - conv1d[2])/conv1d[3])+1
output = (input[0], conv1d[1], int(n_output))
output

(64, 7, 98)

In [379]:
input = (64, 7, 49)
conv1d = (7, 33, 3, 2)
n_output = np.trunc((input[2] - conv1d[2])/conv1d[3])+1
output = (input[0], conv1d[1], int(n_output))
output

(64, 33, 24)

# 모델링

In [380]:
'''
eight of size [33, 7, 3], expected input[64, 3, 49] to have 7 channels, but got 3 channels instead
 weight of size [33, 7, 3], expected input[64, 3, 49] to have 7 channels, but got 3 channels instead
'''

class TextModel(nn.Module):
    def __init__(self):
        super(TextModel, self).__init__()
        self.conv1 = nn.Conv1d(1, 7, 3, 1)
        self.conv2 = nn.Conv1d(7, 33, 3, 2)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(33*12, 128)
        self.fc2 = nn.Linear(128, num_classses)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        # x = F.relu(self.conv1(x))
        # x = F.relu(self.conv2(x))
        x = x.view(-1, 33*12)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [381]:
model = TextModel()
model.to(device)
    
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=args.learning_rate, weight_decay=args.wd)

In [382]:
# 매 epoch마다 ./ckpt 파일에 모델이 저장됩니다.
# validation dataset 없이 모든 train data를 train하는 방식입니다.
if args.train :
    model.train()
    for epoch in range(args.epochs):
        epoch_loss = 0.0
        for iter, (encoded_text_ids, author) in enumerate(train_dataloader):
            if encoded_text_ids.shape[0] != args.batch_size:
                break
            encoded_text_ids = encoded_text_ids.view(args.batch_size, 1, args.max_len).to(device, dtype=torch.float)
            author= author.to(device, dtype=torch.long)
            pred = model(encoded_text_ids)
            loss = criterion(input=pred, target=author)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.detach().item()
            print('epoch : {0} step : [{1}/{2}] loss : {3}'.format(epoch, iter, len(train_dataloader), loss.detach().item()))
        epoch_loss /= len(train_dataloader)
        print('\nepoch : {0} epoch loss : {1}\n'.format(epoch, epoch_loss))
        torch.save(model.state_dict(), args.model_dir + "epoch_{0:03}.pth".format(epoch))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


RuntimeError: ignored

In [ ]:
# 모든 epoch이 끝난 뒤 test 진행
model.eval()
for iter, encoded_text in enumerate(test_dataloader):
    encoded_text = encoded_text.view(1, 1, args.max_len).to(device, dtype=torch.float)
    pred = model(encoded_text)
    pred = nn.Softmax(dim=1)(pred)
    pred = pred.detach().cpu().numpy()
    print(pred)
    sample_submission.loc[iter:iter, ['0','1','2','3','4']] = pred

sample_submission.to_csv(args.fianl_submission_dir, index=False, encoding = 'utf-8')

In [ ]:
sample_submission